<img src="https://pages.cnpem.br/capsuladaciencia/wp-content/uploads/sites/155/2022/10/Ilum.png" alt="Ilum - Escola de Ciência" width="200"/>

**Redes Neurais e Algoritmos Genéticos 2025**

**Prof. Dr. Daniel R. Cassar**

Rafael Dalacorte Erdmann (24017)

## Fera Formidável 12: Novos Palíndromos

**Objetivo:** Encontre pelo menos 10 palíndromos diferentes de 5 letras. Estes palíndromos devem ter pelo menos uma vogal. Não é necessário que eles formem palavras válidas em português ou qualquer outro idioma.

**Dica:** Nada te impede de inventar um novo operador de mutação. Existe uma forma de melhorar bastante a convergência deste problema com um operador de mutação especializado para esta tarefa.

**Comentário:** o código deve de alguma forma gerar os 10 palíndromos diferentes, mas ninguém disse que eles devem ser encontrados na mesma evolução de um algoritmo genético. Quem sabe evoluir um algoritmo mais de uma vez possa ser uma estratégia válida.

### Introdução

Um palíndromo é uma palavra, frase ou número com simetria, de modo que a sua leitura é idêntica da esquerda para direita ou direita para esquerda [1]. Essa simetria pode ser muito útil para reduzir a quantidade de computações necessárias para, por exemplo gerar um novo palíndromo: se o palíndromo final deve ter $9$ letras, deve-se apenas gerar $5$ e espelhar as $4$ primeiras ao final.

Os algoritmos genéticos que utilizamos até o momento, porém, não implementam esse tipo de simetria, de modo que não são eficientes para encontrar palavras palíndromas. Com esse intuito, devemos preparar operadores específicos para a simetria.

### Resolução

Primeiramente, vamos às explicações dos novos operadores específicos do problema:

### Função objetivo para palíndromos

A função objetivo utilizada aqui possui algumas características especiais:
- Ela invalida indivíduos que não possuem vogal ou não são palíndromos (associando-os a um *loss* infinito)
- Ela penaliza indivíduos que já tenham sido observados anteriormente (e salva os indivíduos inéditos dentro do *set* `vistos`)

### Função cruzamento de ponto duplo para palíndromos

O cruzamento de indivíduos palíndromos deve gerar indivíduos palíndromos, de modo que esse operador precisou ser simétrico. De maneira resumida, ele é um cruzamento de ponto duplo, mas o segundo ponto de corte não é aleatório, é simétrico ao primeiro ponto (que é escolhido aleatoriamente).

Ex: Z*OAO*Z U**HZH**U -> Z**HZH**Z U*OAO*U

### Função mutação simples para palíndromos

A mesma necessidade de manter os indivíduos como palíndromos surge na função mutação, que deve ser simétrica. A função escolhida para ser utilizada foi a mutação simples, que troca um gene (nesse caso, sendo um indivíduo simétrico, troca dois genes simétricos em relação ao centro da palavra) por outro dentre os possíveis.

Ex: **U**OAO**U** -> **Z**OAO**Z**

### Código

Começamos importando as funções que serão utilizadas:

In [25]:
from random import seed 
from string import ascii_uppercase
from functools import partial
from pprint import pprint

from funcoes_feras import populacao_senha as cria_populacao
from funcoes_feras import funcao_objetivo_pop_palindromo as funcao_objetivo
from funcoes_feras import selecao_torneio_min as funcao_selecao
from funcoes_feras import cruzamento_ponto_duplo_palindromo as funcao_cruzamento
from funcoes_feras import mutacao_simples_palindromo 

Os parâmetros para o algoritmo genético serão os seguintes:

In [18]:
SENHA_ALEATORIA = 42
seed(SENHA_ALEATORIA)

CARACTERES_POSSIVEIS = ascii_uppercase

TAMANHO_PALINDROMO = 5

NUM_ITERACOES = 2500
TAMANHO_POPULACAO = 100
CHANCE_DE_CRUZAMENTO = 0.25
CHANCE_DE_MUTACAO = 1.0
TAMANHO_TORNEIO = 2

Para a geração da população inicial, poder-se-ia preparar um operador específico que já gere palíndromos, porém nesse caso apenas utilizei o criador de populações utilizado no problema da senha (pois gera uma sequência de caracteres com base em um tamanho desejado e uma lista de caracteres possíveis).

In [19]:
populacao = cria_populacao(TAMANHO_POPULACAO, TAMANHO_PALINDROMO, CARACTERES_POSSIVEIS)
funcao_mutacao = partial(mutacao_simples_palindromo, valores_possiveis=list(CARACTERES_POSSIVEIS))

for i in range(len(populacao)):
    print (i, "".join(populacao[i]))

0 UDAXI
1 HHEXD
2 VXRCS
3 NBACG
4 HQTAR
5 GWUWR
6 NHOSI
7 ZAYZF
8 WNKIE
9 GYKDC
10 MDLLT
11 IZBXO
12 RDMCR
13 JUTLS
14 GWCBV
15 HYJCH
16 DMIOU
17 LFLLG
18 VIWVU
19 CTUFR
20 XHFOM
21 IUWRH
22 VKYYB
23 HBZKM
24 ICGSW
25 KGUPM
26 UOEIE
27 HXRRI
28 XSNSM
29 LHEQP
30 CYBDE
31 UFZVN
32 TCMMT
33 OQIRA
34 VXDVR
35 YIYUK
36 DJNFO
37 AXXIQ
38 YFQDU
39 JUQTG
40 ELYFR
41 YQATK
42 PADLZ
43 JHBHS
44 CCXPC
45 YRYEE
46 VPRFI
47 QTNGR
48 YXWGW
49 JMVUL
50 OQODH
51 HCKAS
52 RHSHA
53 CWUBH
54 CBKCQ
55 HIVPG
56 REXSS
57 PHZPZ
58 NGDDV
59 NLNNO
60 XBVUU
61 DBMXK
62 ZDHGG
63 ROENF
64 IOHCO
65 ZRDBU
66 RACYH
67 FNPPG
68 MBFMA
69 MIZZO
70 JNWXZ
71 RVWPE
72 GJGBS
73 XRBXK
74 BBSPQ
75 QFBQC
76 FCTCV
77 HMDSH
78 STBTC
79 NVSSQ
80 KIGVW
81 KHIME
82 VUJOK
83 YCAOT
84 SDCRG
85 QIELC
86 HLJFO
87 RWJTZ
88 UQAVR
89 JVDEI
90 DDXRE
91 IJTGW
92 KGVUI
93 QPIBC
94 UNIBA
95 KYEUI
96 FXORW
97 NRADC
98 WERBL
99 SRENE


In [20]:
menor_fitness_geral = float("inf")
geracao = 0
vistos = set()
hall_da_fama = set()

fitness = funcao_objetivo(populacao, vistos)
for indice in range(len(fitness)):
    if fitness[indice] == 0:
        hall_da_fama.add("".join(populacao[indice]))

for _ in range(NUM_ITERACOES):
    # Seleção
    fitness = funcao_objetivo(populacao, vistos)
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)
    
    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)
    
    # Mutação
    funcao_mutacao(populacao, CHANCE_DE_MUTACAO)
    
    # Encerramento
    populacao = proxima_geracao
    geracao += 1
    
    fitness = funcao_objetivo(populacao, vistos)
    for indice in range(len(fitness)):
        if fitness[indice] == 0:
            hall_da_fama.add("".join(populacao[indice]))

In [21]:
len(hall_da_fama)

8315

Com esses parâmetros, então, geramos um total de 8315 palíndromos novos, sendo eles:

In [22]:
pprint(hall_da_fama)

{'AAAAA',
 'AABAA',
 'AACAA',
 'AADAA',
 'AAEAA',
 'AAFAA',
 'AAGAA',
 'AAHAA',
 'AAIAA',
 'AAJAA',
 'AAKAA',
 'AALAA',
 'AAMAA',
 'AANAA',
 'AAOAA',
 'AAPAA',
 'AAQAA',
 'AARAA',
 'AASAA',
 'AATAA',
 'AAUAA',
 'AAVAA',
 'AAWAA',
 'AAXAA',
 'AAYAA',
 'AAZAA',
 'ABABA',
 'ABBBA',
 'ABCBA',
 'ABDBA',
 'ABEBA',
 'ABFBA',
 'ABGBA',
 'ABHBA',
 'ABIBA',
 'ABJBA',
 'ABKBA',
 'ABLBA',
 'ABMBA',
 'ABNBA',
 'ABOBA',
 'ABPBA',
 'ABQBA',
 'ABRBA',
 'ABSBA',
 'ABTBA',
 'ABUBA',
 'ABVBA',
 'ABWBA',
 'ABXBA',
 'ABYBA',
 'ABZBA',
 'ACACA',
 'ACBCA',
 'ACCCA',
 'ACDCA',
 'ACECA',
 'ACFCA',
 'ACGCA',
 'ACHCA',
 'ACICA',
 'ACJCA',
 'ACKCA',
 'ACLCA',
 'ACMCA',
 'ACNCA',
 'ACOCA',
 'ACPCA',
 'ACQCA',
 'ACRCA',
 'ACSCA',
 'ACTCA',
 'ACUCA',
 'ACVCA',
 'ACWCA',
 'ACXCA',
 'ACYCA',
 'ACZCA',
 'ADADA',
 'ADBDA',
 'ADCDA',
 'ADDDA',
 'ADEDA',
 'ADFDA',
 'ADGDA',
 'ADHDA',
 'ADIDA',
 'ADJDA',
 'ADKDA',
 'ADLDA',
 'ADMDA',
 'ADNDA',
 'ADODA',
 'ADPDA',
 'ADQDA',
 'ADRDA',
 'ADSDA',
 'ADTDA',
 'ADUDA',
 'ADVDA',


Mas quantos palíndromos com esse número de letras são possíveis? Para essas condições, palíndromos que devem conter uma vogal, isso é fácil de ser calculado através do produto cartesiano (todas as possíveis sequências) dos caracteres possíveis:

In [23]:
from itertools import product

todos_palind = []

for palindromo in product(CARACTERES_POSSIVEIS,repeat= (int(TAMANHO_PALINDROMO/2) + 1)):
    vogal = False

    for letra in ['A','E','I','O','U']:
        if letra in palindromo:
            vogal = True
            break

    if vogal:
        todos_palind.append(list(palindromo))

for palind in todos_palind:
    para_repetir = len(palind)-1
    for i in range(-para_repetir, 0):
        palind.append(palind[-i-1])

todos_palindromos = ["".join(palind) for palind in todos_palind]

len(todos_palindromos)

8315

Portanto, o algoritmo genético foi capaz de encontrar todos os palíndromos de 5 letras. Porém, observe que o algoritmo genético se desenvolveu ao longo de $3$ segundos, enquanto o cálculo das possibilidades foi praticamente instantâneo com o método do itertools.

Para palíndromos com mais letras, essa diferença no tempo de processamento é ainda mais destacada. Mas perceba que ainda assim o algoritmo genético tem sua utilidade: caso quiséssemos palíndromos com condições mais específicas, como "sempre que tiver um C ele é adjacente a A ou sempre que tiver um D ele é adjacente a S", pode ser mais fácil implementar as condições dentro da função objetivo do algoritmo genético que dentro do cálculo de possibilidades com produto cartesiano.

Nas condições do enunciado, porém, estamos atirando em uma formiga com um canhão ao utilizar algoritmo genético. Nosso *pipeline* pode ser utilizado para gerar $10$ palíndromos aleatórios sem precisar calcular todos, mas isso ocorre dentro de uma iteração de evolução.

### Conclusão

Foi possível implementar operadores simétricos (de função objetivo, função cruzamento e função mutação) capazes de acelerar a busca de indivíduos válidos pelo algoritmo genético. 

### Referências

[1] Wikipedia. Palíndromo. Disponível em: https://pt.wikipedia.org/wiki/Pal%C3%ADndromo. Acesso em 11/06/2025.